In [9]:
#@title 檢視 GPU
! nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [10]:
#@title 安裝 colorama, ffmpeg

! pip install colorama
! pip install ffmpeg-python

In [2]:
#@title 安裝 DeepFaceLab
!git clone https://github.com/iperov/DeepFaceLab.git

Cloning into 'DeepFaceLab'...
remote: Enumerating objects: 8010, done.
remote: Counting objects: 100% (259/259), done.
remote: Compressing objects: 100% (140/140), done.
remote: Total 8010 (delta 146), reused 178 (delta 117), pack-reused 7751
Receiving objects: 100% (8010/8010), 823.29 MiB | 29.97 MiB/s, done.
Resolving deltas: 100% (5140/5140), done.
Checking out files: 100% (211/211), done.


In [7]:
#@title 建立 workspace
!mkdir -p /content/workspace; mkdir -p /content/workspace/data_src; mkdir -p /content/workspace/data_src/aligned; mkdir -p /content/workspace/data_dst; mkdir -p /content/workspace/data_dst/aligned; mkdir -p /content/workspace/model  
! ls workspace

data_dst  data_src  model


In [11]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
#@title 用youtube-dl下載影片
! curl -L https://yt-dl.org/downloads/latest/youtube-dl -o /usr/local/bin/youtube-dl
! chmod a+rx /usr/local/bin/youtube-dl
src_youtube = 'https://www.youtube.com/watch?v=EKS6mfLtR2s' #@param {type:"string"}
dst_youtube = 'https://www.youtube.com/watch?v=EKS6mfLtR2s'#@param {type:"string"}
src_yt_cmd =  f'youtube-dl -f mp4 {src_youtube}  --output /content/workspace/data_src.mp4'
dst_yt_cmd =  f'youtube-dl -f mp4 {dst_youtube}  --output /content/workspace/data_dst.mp4'
! $src_yt_cmd

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100     3  100     3    0     0     44      0 --:--:-- --:--:-- --:--:--    44
100     3  100     3    0     0     38      0 --:--:-- --:--:-- --:--:--    38
100   645  100   645    0     0   5466      0 --:--:-- --:--:-- --:--:--  5466
100 1794k  100 1794k    0     0  5789k      0 --:--:-- --:--:-- --:--:-- 30.9M
[youtube] EKS6mfLtR2s: Downloading webpage
[download] Destination: /content/workspace/data_src.mp4
[download]   0.4% of 61.07MiB at 48.91KiB/s ETA 21:13
ERROR: Interrupted by user


In [15]:
#@title 從 Google Drive 複製影片到workspace

src_mp4 = '/content/drive/MyDrive/aidataset/deepfacelab/data_src.mp4' #@param {type:"string"}
dst_mp4 = '/content/drive/MyDrive/aidataset/deepfacelab/data_dst.mp4' #@param {type:"string"}
src_cmd=f"cp {src_mp4} /content/workspace/data_src.mp4"  
dst_cmd=f"cp {dst_mp4} /content/workspace/data_dst.mp4" 
! $src_cmd
! $dst_cmd

In [ ]:
#@title 從 data_src 影片擷取人物圖片
# 開始擷取 src
%cd /content/DeepFaceLab/
! python main.py videoed extract-video --input-file /content/workspace/data_src.mp4 --output-dir /content/workspace/data_src/

In [ ]:
#@title 從 data_dst 影片擷取人物圖片
# 開始擷取 dst
%cd /content/DeepFaceLab/
! python main.py videoed extract-video --input-file /content/workspace/data_dst.mp4 --output-dir /content/workspace/data_dst/